In [1]:
import pandas as pd
import geopandas as gpd
import folium
import geopy
from geopy.distance import geodesic
from shapely.geometry import Point
import branca.colormap as cm
from ipywidgets import interact_manual, widgets, VBox, HBox
import ipywidgets as widgets
from sklearn.preprocessing import MinMaxScaler
import zipfile
import os
import fiona
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## navegador que vai usar
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
## dar opções pro navegador de como interagir 
from selenium.webdriver.chrome.options import Options
## conseguir usar teclas do teclado
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup
from time import sleep


## Web Scraping - Google Maps

In [145]:
def coletar_dados_google_maps(data):
    options = Options()
    driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), 
                              options = options)
    options.add_argument("--headless")
    
    ## Define o site a ser buscado
    driver.get("https://www.google.com/maps/place/91%C2%BA+Distrito+Policial+Civil+SP+(CEAGESP)/@-23.5338885,-46.7384013,14.65z/data=!4m14!1m7!3m6!1s0x94cef8a18059472d:0xefe2215388986b40!2s91%C2%BA+Distrito+Policial+Civil+SP+(CEAGESP)!8m2!3d-23.5387266!4d-46.7309397!16s%2Fg%2F1tg87s_g!3m5!1s0x94cef8a18059472d:0xefe2215388986b40!8m2!3d-23.5387266!4d-46.7309397!16s%2Fg%2F1tg87s_g?entry=ttu&g_ep=EgoyMDI1MDIxMi4wIKXMDSoASAFQAw%3D%3D")
    current_url = "https://www.google.com/maps/place/91%C2%BA+Distrito+Policial+Civil+SP+(CEAGESP)/@-23.5338885,-46.7384013,14.65z/data=!4m14!1m7!3m6!1s0x94cef8a18059472d:0xefe2215388986b40!2s91%C2%BA+Distrito+Policial+Civil+SP+(CEAGESP)!8m2!3d-23.5387266!4d-46.7309397!16s%2Fg%2F1tg87s_g!3m5!1s0x94cef8a18059472d:0xefe2215388986b40!8m2!3d-23.5387266!4d-46.7309397!16s%2Fg%2F1tg87s_g?entry=ttu&g_ep=EgoyMDI1MDIxMi4wIKXMDSoASAFQAw%3D%3D"
    data_list = []
    for nome in data:
        search_box = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "searchboxinput"))  # Usando o ID correto
        )
        search_box.clear()
        
    # Enviar o texto desejado para o campo de pesquisa
        search_box.send_keys(nome+", São Paulo, SP")
    
    # Envia a pesquisa pressionando 'Enter'
        search_box.send_keys(Keys.RETURN)
        sleep(10)

        current_url = driver.execute_script("return window.location.href;")
        data_list.append({"nome": nome, "url": current_url})
    df = pd.DataFrame(data_list)

    return df 

In [146]:
# dados = coletar_dados_google_maps(data)

In [48]:
def extrair_lat_lon(url):
    match = re.search(r"/@(-?\d+\.\d+),(-?\d+\.\d+)", url)
    if match:
        return float(match.group(1)), float(match.group(2))
    return None, None  # Retorna valores nulos se não encontrar

# Aplicar a função ao DataFrame
# dados[['lat', 'lon']] = dados['url'].apply(lambda x: pd.Series(extrair_lat_lon(x)))

In [ ]:
# mapa = criar_mapa_sp()
# layer = folium.FeatureGroup(name="Delegacias")
# plot_circlemarkers_one_color(dados, "blue", 5, "nome", "nome", layer)
# layer.add_to(mapa)

In [ ]:
# dados.to_csv("delegacias_dados_geograficos.csv", index=False)


## Web Scraping - Delegacias

In [ ]:
def coletar_dados(tipo_busca, anos_escolhidos):
    options = Options()
    driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), 
                              options = options)
    options.add_argument("--headless")
    
    ## Define o site a ser buscado
    driver.get("https://www.ssp.sp.gov.br/estatistica/dados-mensais")
    
    wait = WebDriverWait(driver, 10)  # Espera até 10 segundos
    if tipo_busca == 1:
        radio_button = driver.find_elements(By.NAME, "optradio")
        radio_button[1].click()
        sleep(3)  
    
    anos = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "form-select")))
    select_anos = Select(anos)
    
    regioes = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'select[formcontrolname="mensalRegiao"]')))
    select_regiao = Select(regioes)
    select_regiao.select_by_visible_text("Capital")
    
    municipio = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'select[formcontrolname="mensalMunicipio"]')))
    select_municipio = Select(municipio)
    select_municipio.select_by_visible_text("São Paulo")
    
    delegacia = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'select[formcontrolname="mensalDistrito"]')))
    select_delegacia = Select(delegacia)
    
    delegacias = [option.text for option in select_delegacia.options if option.text.strip()]
    return delegacias
    sleep(10)
    for ano in anos_escolhidos:
        select_anos.select_by_value(ano)
        sleep(10)
        for option in select_delegacia.options:
                value = option.get_attribute("value")
                if value != "0" and value!="1604":
                    select_delegacia.select_by_value(value)
                    try:
                        WebDriverWait(driver, 10).until(
                            EC.staleness_of(driver.find_element(By.CLASS_NAME, "table-striped"))
                        )
                    except:
                        print("tempo excedido, mas ok")
                      
                    html = driver.page_source
                    soup = BeautifulSoup(html, "html.parser")
                    table = soup.find("table", class_="table-striped")
                    rows = table.find_all("tr")
                    headers = [header.text.strip() for header in rows[0].find_all("th")]
                    data = []
                    for row in rows[1:]:
                        td_columns = row.find_all("td")
                        th_column = row.find_all("th")
                        row_data = [th_column[0].text.strip()]  
                        row_data.extend([col.text.strip() for col in td_columns])
                        # print(row_data)
                        data.append(row_data)
                    df = pd.DataFrame(data, columns = headers)
                    df['ano'] = ano
                    df['delegacia'] = option.text
                    try:
                        df_aux = pd.concat([df_aux, df])
                    except:
                        df_aux = df.copy()
    return df_aux

In [49]:
# anos = ["2021", "2022", "2023", "2024"]
# data = coletar_dados(0,anos)
# data

In [50]:
# data.to_csv('produtividade_policial.csv', index=False)
# data.to_csv('criminalidade', index = False)

In [51]:
# data.to_csv('criminalidade.csv', index=False)
# data.to_csv('produtividade_policial.csv', index=False)


# Funções de Estações

In [52]:
## Prepara o dataframe de estações para as outras análises
## Retorna o dataframe tratado
def tratar_df_estacoes(df_estacoes):
    ## Cria uma geometria "Point"
    df_estacoes = gpd.GeoDataFrame(df_estacoes, geometry = gpd.points_from_xy(df_estacoes['lon'], df_estacoes['lat']))
    ## Define crs
    df_estacoes.set_crs("EPSG:4326", inplace = True)
    return df_estacoes

## Cria circles (geometry Polygon) em volta de cada estação (geometry Point)
## Retorna o dataframe com a geometria "circles"
def criar_circulos_estacao(data, radius_km):
    circles = []
    for _, row in data.iterrows():
        circle_points = []
        center = (row['lat'], row['lon'])
        ## Dizemos que é aproximadamente um círculo, pois é um polígono regular com 360 vértices
        for angle in range(0,360, 1): 
            destination = geopy.distance.distance(kilometers = radius_km).destination(center, angle)
            circle_points.append(Point(destination.longitude, destination.latitude))
        circle = gpd.GeoSeries(circle_points).unary_union.convex_hull
        circles.append(circle)
    data.rename(columns = {'geometry':'point'}, inplace = True)
    circles_gdf = gpd.GeoDataFrame(data, geometry = circles)
    circles_gdf.rename(columns = {'geometry':'circle'}, inplace = True)
    return circles_gdf

## Adiciona os círculos no mapa, usa como base a função "criar_circulos_estacao"
## Retorna o Mapa com os círculos (geometry polygon) e pontos (geometry point) adicionados
def adicionar_estacoes_com_circulos_mapa(mapa, df_estacoes, radius_km):
    ## Definição dos layers para controle do usuário por camadas
    layer_estacoes =  folium.FeatureGroup(name = "Estações")
    layer_circulos_estacoes = folium.FeatureGroup(name = f"Redondezas {radius_km} km") 
    ## Se o df_estacoes já tem os circles, não precisa criá-los
    if "circle" not in df_estacoes:
        data = criar_circulos_estacao(df_estacoes, radius_km)
    else:
        data = df_estacoes.copy()
    ## Define "circle" como geometria a ser usada, pois há mais de uma em uso
    data.set_geometry("circle", inplace = True)
    ## Adiciona Circlemarkers ao layer
    for _, row in data.iterrows():
        folium.CircleMarker(
                    location=[row['lat'],row['lon']],
                    color = "blue",
                    radius= 5,
                    tooltip= row['name'],
                    fill=True,
                    fill_opacity=1,
                    fill_color="blue",
                ).add_to(layer_estacoes)
    ## Adiciona os Circles ao layer
    for _, row in data.iterrows():
        folium.GeoJson(
            row['circle'].__geo_interface__, 
            tooltip = row['name'],
            style_function=lambda x:{'fillColor': 'gray', 'color': 'gray', 'fillOpacity': 0.0},
        ).add_to(layer_circulos_estacoes)
        layer_estacoes.add_to(mapa)
        layer_circulos_estacoes.add_to(mapa)
        folium.LayerControl().add_to(mapa)
    return mapa

# Funções de Mapa

In [53]:
## Define cores para mapas
def get_color(valor):
    if valor == 1:
        return "yellow"
    elif valor == 2:
        return "orange"
    elif valor == 3:
        return "red"
    elif valor == 4:
        return "darkred"
    elif valor == 5:
        return "blue"
    elif valor == 6:
        return "darkblue"
    elif valor == 7:
        return "purple"
    elif valor == 8:
        return "pink"
    elif valor == 9:
        return "green"
    elif valor == 10:
        return "darkgreen"
    elif valor == 11:
        return "lightblue"
    elif valor == 12:
        return "cyan"
    elif valor == 13:
        return "brown"
    elif valor == 14:
        return "gray"
    elif valor == 15:
        return "black"
    elif valor == 16:
        return "lightgray"
    elif valor == 17:
        return "white"
    else:
        return "black"  # Valor inválido

## Cria um mapa centrado em São Paulo
def criar_mapa_sp():
    mapa_sp = folium.Map(location = [-23.550520, -46.633308], zoom_start = 12)
    return mapa_sp

## Plota Circlemarkers de 1 cor, dado 1 df com Lat e Lon
def plot_circlemarkers_one_color(data, color, radius, name, pontuacao, layer):
    for _, row in data.iterrows():
        folium.CircleMarker(
                location = [row['lat'], row['lon']],
                color = color, 
                radius = radius,
                tooltip = row[name] + " " + str(row[pontuacao]), 
                popup = row[name] + " " + str(row[pontuacao]), 
                fill = True,
                fill_opacity = 1,
                fill_color = color,
        ).add_to(layer)

## Plota Circlemarkers de várias cores, dado 1 df com Lat e Lon e um Colormap branca
def plot_circlemarkers_multi_color(data, radius, name, pontuacao, layer, colormap):
    for _, row in data.iterrows():
        folium.CircleMarker(
                location = [row['lat'], row['lon']],
                color = colormap(row[pontuacao]), 
                radius = radius,
                tooltip = row[name] + " " + str(row[pontuacao]), 
                popup =  row[name] + " " + str(row[pontuacao]), 
                fill = True,
                fill_opacity = 1,
                fill_color = colormap(row[pontuacao]),
        ).add_to(layer)
        
    


## Funções de Outliers

In [54]:
## Plota os Circlemarkers outliers dado um out_lower, out_upper e non_outliers, separados pela função "identificar_outliers"
def plot_circlemarkers_outliers(out_lower, out_upper, non_outliers, color_lower, color_upper, radius, 
                                name, pontuacao, layer, colormap):
    plot_circlemarkers_one_color(out_lower, color_lower, radius, name, pontuacao, layer)
    plot_circlemarkers_one_color(out_upper, color_upper, radius, name, pontuacao, layer)
    plot_circlemarkers_multi_color(non_outliers, radius, name, pontuacao, layer, colormap)

def identificar_outliers(df, coluna):
    """
    Identifica outliers em uma coluna de um DataFrame usando o método IQR.

    Parâmetros:
        df (pd.DataFrame): DataFrame contendo os dados.
        coluna (str): Nome da coluna para identificar os outliers.

    Retorna:
        outliers_lower (pd.DataFrame): DataFrame contendo os outliers inferiores.
        outliers_upper (pd.DataFrame): DataFrame contendo os outliers superiores.
        non_outliers (pd.DataFrame): DataFrame contendo os dados não-outliers.
    """
    Q1 = df[coluna].quantile(0.25) 
    Q3 = df[coluna].quantile(0.75)
    IQR = Q3 - Q1
    lowerbound = Q1 - 1.5 * IQR
    upperbound = Q3 + 1.5 * IQR

    outliers_lower = df[df[coluna] < lowerbound]
    outliers_upper = df[df[coluna] > upperbound]
    non_outliers = df[(df[coluna] >= lowerbound) & (df[coluna] <= upperbound)]

    return outliers_lower, outliers_upper, non_outliers 


# Funções de Criminalidade

In [46]:
# mapa = criar_mapa_sp()
# layer = folium.FeatureGroup(name="Delegacias")
# plot_circlemarkers_one_color(dados_geograficos, "blue", 5, "nome", "nome", layer)
# layer.add_to(mapa)
# mapa

In [115]:
## Prepara o dataframe de ocorrencias para as outras análises
## Retorna o dataframe tratado
def gerar_df_criminalidade(dados_geograficos_aux):
    dados_geograficos = dados_geograficos_aux.copy()
    criminalidade = pd.read_csv('criminalidade.csv')
    produtividade_policial = pd.read_csv('produtividade_policial.csv')

    crimes_totais = pd.concat([criminalidade, produtividade_policial])
    meses_rename = {
        "Jan": 1, "Fev": 2, "Mar": 3, "Abr": 4, "Mai": 5, "Jun": 6,
        "Jul": 7, "Ago": 8, "Set": 9, "Out": 10, "Nov": 11, "Dez": 12
    }
    crimes_totais.rename(columns=meses_rename, inplace=True)
    
    crimes_totais_new = crimes_totais.melt(
        id_vars=["Natureza", "ano", "delegacia"],
        value_vars=list(range(1, 13)),  
        var_name="mes",
        value_name="num_ocorrencias"
    )
    crimes_totais_new["data"] = pd.to_datetime(crimes_totais_new["ano"].astype(str) + "-" + crimes_totais_new["mes"].astype(str) + "-01")

    dados_geograficos.loc[dados_geograficos['nome'] == "9ª DDM - 3ª Seccional - Oeste", 'lat'] = -23.4818389
    dados_geograficos.loc[dados_geograficos['nome'] == "9ª DDM - 3ª Seccional - Oeste", 'lon'] = -46.7258414
    dados_geograficos = dados_geograficos.drop_duplicates()
    dados_geograficos = dados_geograficos[~dados_geograficos['nome'].isin([
        "Del. Aeroporto Int. Viracopos - CPS (SP)",
        "Delegacia Porto de Santos (SP)",
        "Delegacias"
    ])]
    dados_geograficos = dados_geograficos.rename(columns={'nome':'delegacia'})
    df = pd.merge(crimes_totais_new, dados_geograficos, on = "delegacia", how = "left")
    geometry = [Point(xy) for xy in zip(df["lon"], df["lat"])]
    gdf_final = gpd.GeoDataFrame(df, geometry=geometry)
    return gdf_final
    
def analise_criminalidade(df_criminalidade, df_estacoes,dados_geograficos, radius_km, natureza, data_min, data_max):
    data_min = pd.to_datetime(data_min)
    data_max = pd.to_datetime(data_max)
    mapa = criar_mapa_sp()
    df_criminalidade = df_criminalidade[(df_criminalidade["data"] >= data_min) & (df_criminalidade["data"] <= data_max)]
    
    df_estacoes_circulo = criar_circulos_estacao(df_estacoes, radius_km)
    df_estacoes_circulo.set_geometry("circle", inplace=True)  

    df_estacoes_circulo.set_crs("EPSG:4326", inplace = True)
    ## Quais ocorrencias eu vou considerar?
    df_criminalidade_filtrado = df_criminalidade[df_criminalidade['Natureza'].isin(natureza)]
    df_criminalidade_filtrado = df_criminalidade_filtrado.groupby('delegacia').agg(
        num_ocorrencias=('num_ocorrencias', 'sum'),
        geometry=('geometry', 'first')  # ou 'centroid' dependendo da sua necessidade
    ).reset_index()
    df_criminalidade_filtrado = gpd.GeoDataFrame(df_criminalidade_filtrado, geometry='geometry')
    df_criminalidade_filtrado.set_crs("EPSG:4326", inplace = True)

    cruzamentos =  gpd.sjoin(df_estacoes_circulo, df_criminalidade_filtrado, how="inner", predicate="intersects")
    
    cruzamentos = cruzamentos.merge(
        df_criminalidade[["delegacia", "geometry"]],  # Apenas as colunas relevantes
        on="delegacia",  # A coluna para equivalência
        how="left"  # Tipo de junção (left mantém todos os dados de cruzamentos)
    )
    ocorrencias_necessarias_para_plot = cruzamentos.copy()
  
    layer_estacoes = folium.FeatureGroup(name = "Estações")
    layer_circulos_estacoes = folium.FeatureGroup(name = f"Redondezas {radius_km} km") 
    layer_criminalidade = folium.FeatureGroup(name="Delegacias")
    plot_circlemarkers_one_color(dados_geograficos, "blue", 5, "nome", "url", layer_criminalidade)

    cruzamentos = cruzamentos.groupby('name', as_index = False)['num_ocorrencias'].count()
    cruzamentos= pd.merge(cruzamentos, df_estacoes_circulo, on = 'name',  how = 'right').fillna(0)
    # scaler = MinMaxScaler()
    # cruzamentos['num_ocorrencias'] = scaler.fit_transform(cruzamentos[['num_ocorrencias']])
    out_lower, out_upper, non_outliers = identificar_outliers(cruzamentos, "num_ocorrencias")
    colormap = cm.linear.plasma.scale(non_outliers['num_ocorrencias'].min(), 
                                      non_outliers['num_ocorrencias'].max())
    colormap.add_to(mapa)

    plot_circlemarkers_outliers(out_lower, out_upper, non_outliers, "black", "green", 5, 
                                "name", "num_ocorrencias", layer_estacoes, colormap)  

    ## Plota círculos em volta das estações
    for _, row in df_estacoes_circulo.iterrows():    
        folium.GeoJson(
            row['circle'].__geo_interface__, 
            tooltip = row['name'],
            style_function=lambda x:{'fillColor': 'gray', 'color': 'gray', 'fillOpacity': 0.0},
        ).add_to(layer_circulos_estacoes)
        
    layer_criminalidade.add_to(mapa)
    layer_estacoes.add_to(mapa)
    layer_circulos_estacoes.add_to(mapa)
    folium.LayerControl().add_to(mapa)
    return mapa
def roda_analise_criminalidade(df_criminalidade, df_estacoes, dados_geograficos):
    min_data = df_criminalidade['data'].min()
    max_data = df_criminalidade['data'].max()
    im = interact_manual(
        lambda radius_km, natureza, data_min, data_max: analise_criminalidade(df_criminalidade,
                                                        df_estacoes, dados_geograficos, radius_km, natureza, data_min, data_max),
        radius_km=widgets.FloatSlider(value=0.5,  min=0.1, max=5),
        natureza=widgets.SelectMultiple(options=[classe for classe in df_criminalidade['Natureza'].unique()]),
        data_min= widgets.DatePicker(
            value=min_data,
            min_date=min_data,
            max_date=max_data
        ),
        data_max= widgets.DatePicker(
            value=max_data,
            min_date=min_data,
            max_date=max_data
        ),
    )
    im.widget.children[0].description = 'Raio Km'
    im.widget.children[1].description = 'Tipo de Crime'
    im.widget.children[2].description = 'Data Mínima'
    im.widget.children[3].description = 'Data Máxima'

In [107]:
estacoes = pd.read_parquet("estacoes_reestruturadas.parquet")
dados_geograficos = pd.read_csv("delegacias_dados_geograficos.csv")

In [108]:
df_criminalidade = gerar_df_criminalidade(dados_geograficos)
df_estacoes = tratar_df_estacoes(estacoes)

In [116]:
roda_analise_criminalidade(df_criminalidade, df_estacoes, dados_geograficos)

interactive(children=(FloatSlider(value=0.5, description='radius_km', max=5.0, min=0.1), SelectMultiple(descri…